In [1]:
from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    set_global_service_context,
)
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.1", request_timeout= 360.0)


In [2]:
# Reads pdfs at "./" path
documents = (
    SimpleDirectoryReader(
        input_files= ['data.pdf'])
        .load_data()
)

In [3]:
documents

[Document(id_='b1ce50fe-3e43-4653-9c4c-d04099609ce8', embedding=None, metadata={'page_label': '1', 'file_name': 'data.pdf', 'file_path': 'data.pdf', 'file_type': 'application/pdf', 'file_size': 18223, 'creation_date': '2024-08-15', 'last_modified_date': '2024-08-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Ram: Sita, have you been following the recent geopolitical shifts? The world seems \nto be in constant flux, with new alliances forming and old ones breaking apart.\nSita: Yes, Ram. It’s fascinating, yet concerning. The balance of power is always \nshifting, and with it, the dynamics of peace and conflict. What do you make of the \ngrowing tensions in the Indo-Pacific region?\nRam: The Indo-Pacific is becoming a key battleground for influence,

In [4]:

# ServiceContext is a bundle of commonly used 
# resources used during the indexing and 
# querying stage 
service_context = (
    ServiceContext
    .from_defaults(
        llm=llm, 
        embed_model="local:BAAI/bge-small-en-v1.5", 
        chunk_size=300
    )
)
set_global_service_context(service_context)

# Node represents a “chunk” of a source Document
nodes = (
    service_context
    .node_parser
    .get_nodes_from_documents(documents)
)

# offers core abstractions around storage of Nodes, 
# indices, and vectors
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# Create the vectorstore index
index = (
    VectorStoreIndex
    .from_documents(
        documents, 
        storage_context=storage_context, 
        llm=llm
        )
)


/tmp/ipykernel_56566/2018639152.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  .from_defaults(
/home/rajatdulal/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Query Engine

In [6]:
query_engine = index.as_query_engine(similarity_top_k=3)
# query_engine = index.as_query_engine()

In [7]:
query = "What does Ram think of Abraham Accords?"

response = query_engine.query(query)

In [8]:
print(response)

Ram thinks that The Abraham Accords have certainly shifted the dynamics in the Middle East. While they’ve opened the door to new economic opportunities, they’ve also created new tensions, especially with Iran.


In [9]:
response.response

'Ram thinks that The Abraham Accords have certainly shifted the dynamics in the Middle East. While they’ve opened the door to new economic opportunities, they’ve also created new tensions, especially with Iran.'

In [10]:
for node in response.source_nodes:
    print(f"{node.get_score()} 👉 {node.text}")

0.6445256790996351 👉 Ram: The Indo-Pacific is becoming a key battleground for influence, especially with  
the rise of China. Their assertive stance in the South China Sea, coupled with 
their Belt and Road Initiative, is reshaping regional power structures. Nations 
like India, Japan, and Australia are responding by strengthening their own ties, 
seeking to counterbalance China’s influence.
Sita: It’s a delicate dance, isn’t it? While economic partnerships are crucial, 
they also need to ensure that these alliances don’t escalate into conflicts. The 
Quad, for instance, is an interesting development. But can these nations maintain a  
balance between economic cooperation and strategic deterrence?
Ram: That’s the challenge. The Quad—comprising India, the U.S., Japan, and 
Australia—does have the potential to be a stabilizing force. However, they must 
navigate this without provoking China too much. The goal should be peace through 
strength, not through aggression.
Sita: Indeed, but it

# Chat Engine

In [5]:
query = "Which countries are trying to counter China?"
chat_eng = index.as_chat_engine(similarity_top_k=3, chat_mode='context')
response = chat_eng.chat(query)

In [6]:
print(response)

According to the provided text, nations like India, Japan, and Australia are responding to China's rise by strengthening their own ties, seeking to counterbalance China's influence. This is mentioned in the following sentences:

"...Nations like India, Japan, and Australia are responding by strengthening their own ties, seeking to counterbalance China’s influence."

and

"Ram: That’s the challenge. The Quad—comprising India, the U.S., Japan, and Australia—does have the potential to be a stabilizing force."


In [7]:
query = "What is the reason behind such counterbalance?"
chat_eng = index.as_chat_engine(similarity_top_k=3, chat_mode='context')
response = chat_eng.chat(query)  

In [8]:
response.response

"The reason behind the counterbalance in the Indo-Pacific region, as well as other regions like the Middle East and Eastern Europe, is primarily driven by security concerns and a desire to maintain strategic deterrence.\n\nChina's growing economic and military influence has led countries like India, Japan, Australia, and others to strengthen their own ties and form alliances to counterbalance China's power. This is motivated by several factors:\n\n1. **Security concerns**: The rise of China's military capabilities and its assertive behavior in the South China Sea have raised concerns among regional nations about potential threats to their sovereignty and territorial integrity.\n2. **Fear of Chinese dominance**: The growing economic and strategic influence of China has led some countries to worry that they will become too dependent on China, compromising their national interests and autonomy.\n3. **Competition for resources and markets**: The increasing economic competition between Chin

In [9]:
for node in response.source_nodes:
    print(f"{node.get_score()} 👉 {node.text}")

0.6069744714179234 👉 Ram: Sita, have you been following the recent geopolitical shifts? The world seems 
to be in constant flux, with new alliances forming and old ones breaking apart.
Sita: Yes, Ram. It’s fascinating, yet concerning. The balance of power is always 
shifting, and with it, the dynamics of peace and conflict. What do you make of the 
growing tensions in the Indo-Pacific region?
Ram: The Indo-Pacific is becoming a key battleground for influence, especially with  
the rise of China. Their assertive stance in the South China Sea, coupled with 
their Belt and Road Initiative, is reshaping regional power structures. Nations 
like India, Japan, and Australia are responding by strengthening their own ties, 
seeking to counterbalance China’s influence.
Sita: It’s a delicate dance, isn’t it? While economic partnerships are crucial, 
they also need to ensure that these alliances don’t escalate into conflicts. The 
Quad, for instance, is an interesting development. But can these na